In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from joblib import dump, load
from auto_tsmo import full_method
from filter_big import bFilter, bPostProcessing
from filter_med import mFilter, mPostProcessing
from filter_small import sFilter, sPostProcessing
from drawing import showRect, showRects, showContours
from feature_extraction import getFeatures, getFeaturesVector

video_example = r'C:\Users\admin\Google\HumanDetection\Data\example.mp4'
video_path = r'C:\Users\admin\Google\HumanDetection\Data\video.mp4'

big_dataset = r"C:\Users\admin\Google\HumanDetection\Final\big_dataset"
med_dataset = r"C:\Users\admin\Google\HumanDetection\Final\med_dataset"
small_dataset = r"C:\Users\admin\Google\HumanDetection\Final\small_dataset"

In [3]:
small_model = load('svm_small_hog_lbp.joblib')
small_add_LBP = True
small_add_HOG = True

def sClassification(gray, rects):
    true_rects = []
    for rect in rects:
        x,y,w,h = rect
        if bool(small_model.predict(getFeatures(gray[y:y+h,x:x+w], small_add_HOG, small_add_LBP))):
            true_rects.append(rect)
    return true_rects

In [4]:
med_model = load('svm_med_hog_lbp.joblib')
med_add_LBP = True
med_add_HOG = False

def mClassification(gray, rects):
    true_rects = []
    for rect in rects:
        x,y,w,h = rect
        if bool(med_model.predict(getFeatures(gray[y:y+h,x:x+w], med_add_HOG, med_add_LBP))):
            true_rects.append(rect)
    return true_rects

In [5]:
big_model = load('svm_big_hog_lbp.joblib')
big_add_LBP = False
big_add_HOG = True

def bClassification(gray, rects):
    true_rects = []
    for rect in rects:
        x,y,w,h = rect
        if bool(big_model.predict(getFeatures(gray[y:y+h,x:x+w], big_add_HOG, big_add_LBP))):
            true_rects.append(rect)
    return true_rects

In [9]:
cap = cv2.VideoCapture(video_path)

start = time.time()

first_frame = 0
curr = 0

while(cap.isOpened()):
    curr += 1
    
    ret, frame = cap.read()
    
    if (curr < first_frame):
        start = time.time()
        continue
    
    frame = frame[10:-10,40:-40]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    thresholds, masks = full_method(gray)
    
    contours = sFilter(thresholds, masks)
    rects = sPostProcessing(contours)
    #showRects(frame, rects, 'red')  
    rects = sClassification(gray, rects)
    showRects(frame, rects)
    
    contours = mFilter(thresholds, masks)
    rects = mPostProcessing(contours)
    #showRects(frame, rects, 'red')
    rects = mClassification(gray, rects)
    showRects(frame, rects)
    
    contours = bFilter(thresholds, masks)
    rects = bPostProcessing(contours)
   # showRects(frame, rects, 'red')
    rects = bClassification(gray, rects)
    showRects(frame, rects)

        
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
end = time.time()

print("Average FPS:", (curr-first_frame)/(end-start))

cap.release()
cv2.destroyAllWindows()

Average FPS: 2.6450872719121388
